In [1]:
import pandas as pd

# Replace 'path_to_your_file' with the actual path where you saved the Crimes.csv file
file_path = 'weather.csv'

# Read the CSV file into a DataFrame
weather = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(weather.head())


                    name    datetime  tempmax  tempmin  temp  feelslikemax  \
0  Chicago,United States  2010-01-01     -8.3    -14.5 -11.7         -15.0   
1  Chicago,United States  2010-01-02    -11.4    -16.4 -14.1         -18.4   
2  Chicago,United States  2010-01-03     -7.9    -17.5 -12.5         -14.7   
3  Chicago,United States  2010-01-04     -7.4    -13.1 -10.2         -14.4   
4  Chicago,United States  2010-01-05     -5.1     -9.5  -7.9         -11.7   

   feelslikemin  feelslike   dew  humidity  ...  solarenergy  uvindex  \
0         -22.4      -19.0 -16.1      69.8  ...          6.6        3   
1         -25.3      -22.6 -19.6      63.1  ...          8.5        4   
2         -25.6      -20.5 -17.6      66.0  ...          7.6        4   
3         -21.3      -17.6 -14.6      69.8  ...          3.5        2   
4         -15.9      -14.2 -12.3      70.8  ...          5.8        3   

   severerisk              sunrise               sunset  moonphase  \
0         NaN  2010-01